In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_news(url, source):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = []
    
    if source == 'us':
        # 以CNBC为例
        articles = soup.select('.Card-titleContainer')
        for article in articles:
            title = article.select_one('.Card-title').text.strip()
            link = 'https://www.cnbc.com' + article.select_one('a')['href']
            news_items.append({'title': title, 'link': link, 'source': 'US'})
    elif source == 'tw':
        # 以台灣經濟日報为例
        articles = soup.select('.block_title')
        for article in articles:
            title = article.text.strip()
            link = 'https://money.udn.com' + article.find('a')['href']
            news_items.append({'title': title, 'link': link, 'source': 'TW'})
    
    return pd.DataFrame(news_items)

try:
    us_news = fetch_news('https://www.cnbc.com/world/?region=world', 'us')
    tw_news = fetch_news('https://money.udn.com/money/cate/5591', 'tw')
    all_news = pd.concat([us_news, tw_news])
    print(f"获取到 {len(all_news)} 条新闻")
except Exception as e:
    print(f"获取新闻时出错: {str(e)}")

获取到 28 条新闻


## Bing Chat

In [5]:
from textblob import TextBlob

def sentiment_analysis(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

all_news['sentiment'] = all_news['title'].apply(sentiment_analysis)

print(all_news[['title', 'sentiment']].head())
print(f"平均情感得分: {all_news['sentiment'].mean()}")

                                               title  sentiment
0  Stock futures are little changed as Wall Stree...  -0.187500
1  Stocks making the biggest moves after hours: I...   0.500000
2  U.S. oil holds to gains after three weeks of c...  -0.700000
3  'Get Britain building again': New UK finance c...   0.136364
4  Britain's Labour pulled off a thumping electio...   0.000000
平均情感得分: 0.0642361111111111


In [6]:
def combine_analysis(ticker, stock_data, news_data):
    stock_analysis = stock_data.get(ticker, {})
    relevant_news = news_data[news_data['title'].str.contains(ticker, case=False)]
    avg_sentiment = relevant_news['sentiment'].mean() if not relevant_news.empty else 0
    
    # 这里的权重可以根据实际情况调整
    technical_score = stock_analysis.get('additional_win_rate', 50) / 100
    combined_score = (technical_score * 0.7 + avg_sentiment * 0.3)
    return combined_score

# 假设 STOCKS 是你要分析的股票列表
for ticker in STOCKS:
    combined_score = combine_analysis(ticker, stock_data, all_news)
    print(f"{ticker} 综合得分: {combined_score:.2f}")

NameError: name 'STOCKS' is not defined